In [1]:
# essential to initialize
import findspark
findspark.init()
# findspark.find()

In [2]:
# multiple ways to initialize sc
from pyspark import SparkContext
from pyspark.sql import SparkSession
sc = SparkContext('local[16]','First App')

In [3]:
#spark = SparkSession(sc)

In [4]:
# reviews = spark.read.json('asnlib/publicdata/review.json')

In [5]:
# reviews.first()

In [6]:
# reviews_large = spark.read.json('asnlib/publicdata/review_large.json')

In [7]:
# reviews_large.first()

In [8]:
reviews_txt = sc.textFile('asnlib/publicdata/review.json')

In [9]:
# reviews_txt.collect()

In [10]:
reviews_txt.first()

'{"review_id": "-I5umRTkhw15RqpKMl_o1Q", "user_id": "-mA3-1mN4JIEkqOtdbNXCQ", "business_id": "mRUVMJkUGxrByzMQ2MuOpA", "stars": 1.0, "text": "Walked in around 4 on a Friday afternoon, we sat at a table just off the bar and walked out after 5 min or so. Don\'t even think they realized we walked in. However everyone at the bar noticed we walked in!!! Service was non existent at best. Not a good way for a new business to start out. Oh well, the location they are at has been about 5 different things over the past several years, so they will just be added to the list. SMDH!!!", "date": "2017-12-15 23:27:08"}'

In [11]:
import json

reviews_json = reviews_txt.map(lambda x: json.loads(x))
reviews_json.first()['date']

'2017-12-15 23:27:08'

In [12]:
reviews_json.count()

1151625

In [13]:
reviews_json.filter(lambda x: '2012' in x['date']).count()

63540

In [14]:
reviews_json.filter(lambda x: '2018' in x['date']).count()

202784

In [ ]:
reviews_json.map(lambda x: (x['date'][0:4], 1)).reduceByKey(lambda a,b: a+b).sortByKey(ascending=False).collect()

In [ ]:
reviews_json.map(lambda x: x['user_id']).distinct().count()

In [ ]:
user_review_count = reviews_json.map(lambda x: (x["user_id"], 1)).reduceByKey(lambda a, b: a + b)

In [ ]:
# user_review_count.collect()

In [ ]:
user_review_count.count() == reviews_json.map(lambda x: x['user_id']).distinct().count()

In [ ]:
user_review_count_sorted = user_review_count.sortBy(lambda x: x[1], ascending=False)

In [ ]:
# user_review_count_sorted.collect()

In [ ]:
user_review_count_sorted.take(5)

In [15]:
reviews_rdd = reviews_json.map(lambda x: (x['text']))
reviews_rdd.first()

"Walked in around 4 on a Friday afternoon, we sat at a table just off the bar and walked out after 5 min or so. Don't even think they realized we walked in. However everyone at the bar noticed we walked in!!! Service was non existent at best. Not a good way for a new business to start out. Oh well, the location they are at has been about 5 different things over the past several years, so they will just be added to the list. SMDH!!!"

In [16]:
punctuations = sc.parallelize(["(", "[", ",", ".", "!", "?", ":", ";", "]", ")"])
stopwords = sc.textFile('asnlib/publicdata/stopwords')

In [17]:
punctuations.first()

'('

In [18]:
stopwords.first()

'i'

In [19]:
all_stopwords = stopwords.union(punctuations)

In [ ]:
all_stopwords.collect()

In [20]:
words_count_rdd = reviews_rdd.flatMap(lambda line: line.split(" ")).map(lambda x: (x.lower().strip(),1)).reduceByKey(lambda a,b: a+b)


In [25]:
words_count_rdd.take(50)

[('even', 216815),
 ('they', 1006809),
 ('noticed', 18335),
 ('not', 747774),
 ('for', 1494188),
 ('from', 352839),
 ('sure', 100247),
 ('fresh', 76030),
 ('there', 391941),
 ('poland', 29),
 ('part', 36730),
 ('sell', 11424),
 ('buy', 28193),
 ('', 2009961),
 ('prices', 55682),
 ('wait', 98647),
 ('still', 111437),
 ('almost', 47371),
 ('restaurant,', 11565),
 ('which', 257043),
 ('before', 108397),
 ('filling,', 1352),
 ('well', 113245),
 ('fantastic!', 4509),
 ('flip', 1333),
 ('know', 122268),
 ('ever', 97012),
 ('elna', 2),
 ('took', 108656),
 ('waitress', 39585),
 ('card/', 7),
 ('away', 45408),
 ('toss', 850),
 ('phone', 31881),
 ('friday,', 1250),
 ("let's", 7832),
 ('fits', 1549),
 ('(1)', 381),
 ('literally', 19703),
 ('handle', 5796),
 ('truth', 1105),
 ('gotten', 12047),
 ('department', 4519),
 ('seen', 23764),
 ('places', 48919),
 ('kids', 30008),
 ('coupons.', 491),
 ('gem!', 1456),
 ('living', 9673),
 ('whatsoever', 656)]

In [24]:
words_count_rdd.filter(lambda x: x[0] not in all_stopwords).sortBy(lambda x: x[1], ascending=False).take(10)

Traceback (most recent call last):
  File "D:\programming\spark_home\spark-2.3.2-bin-hadoop2.7\python\pyspark\cloudpickle.py", line 235, in dump
    return Pickler.dump(self, obj)
  File "D:\programming\Anaconda3\lib\pickle.py", line 437, in dump
    self.save(obj)
  File "D:\programming\Anaconda3\lib\pickle.py", line 504, in save
    f(self, obj) # Call unbound method with explicit self
  File "D:\programming\Anaconda3\lib\pickle.py", line 789, in save_tuple
    save(element)
  File "D:\programming\Anaconda3\lib\pickle.py", line 504, in save
    f(self, obj) # Call unbound method with explicit self
  File "D:\programming\spark_home\spark-2.3.2-bin-hadoop2.7\python\pyspark\cloudpickle.py", line 378, in save_function
    self.save_function_tuple(obj)
  File "D:\programming\spark_home\spark-2.3.2-bin-hadoop2.7\python\pyspark\cloudpickle.py", line 529, in save_function_tuple
    save(closure_values)
  File "D:\programming\Anaconda3\lib\pickle.py", line 504, in save
    f(self, obj) # Call

PicklingError: Could not serialize object: Exception: It appears that you are attempting to broadcast an RDD or reference an RDD from an action or transformation. RDD transformations and actions can only be invoked by the driver, not inside of other transformations; for example, rdd1.map(lambda x: rdd2.values.count() * x) is invalid because the values transformation and count action cannot be performed inside of the rdd1.map transformation. For more information, see SPARK-5063.